<h1>RNN Project</h1>

In [74]:
import re 
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer ## stemming 
from nltk.stem import WordNetLemmatizer  ## lemmatization
import joblib,os 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences 
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd

print('libraries imported 😎')

libraries imported 😎


<h1>1. Reading data from text files.</h1>


In [19]:
train_data = open(r"C:\Users\hp\Desktop\RNN\train.txt").readlines()
test_data = open(r"C:\Users\hp\Desktop\RNN\test.txt").readlines()
val_data = open(r"C:\Users\hp\Desktop\RNN\val.txt").readlines()

In [20]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

16000
2000
2000


# 2. Combining multiple list in one 

In [21]:
complete_data = train_data + test_data + val_data 
len(complete_data)

20000

# 3. Split lines into two list

In [22]:
x=[]
y=[]
for temp in complete_data:
    complete = temp.split(';')
    if len(complete)==2:
        x.append(complete[0])
        y.append(complete[1].strip())       

In [23]:
x

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy',
 'ive been feeling a little burdened lately wasnt sure why that was',
 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny',
 'i feel as confused about life as a teenager or as jaded as a year old man',
 'i have been with petronas for years i feel that petronas has performed well and made a huge profit',
 'i feel romantic too',
 'i feel like i have to make the suffering i m seeing mean something',
 'i do feel that running is a divine experience and that i can expect to have some type of spiritual encounter',
 'i think it s the easiest time of year to feel dissatisfied',
 'i feel low energy i m just thirst

In [24]:
y

['sadness',
 'sadness',
 'anger',
 'love',
 'anger',
 'sadness',
 'surprise',
 'fear',
 'joy',
 'love',
 'sadness',
 'joy',
 'anger',
 'sadness',
 'joy',
 'joy',
 'sadness',
 'sadness',
 'sadness',
 'fear',
 'anger',
 'fear',
 'joy',
 'joy',
 'anger',
 'sadness',
 'sadness',
 'sadness',
 'anger',
 'joy',
 'joy',
 'fear',
 'surprise',
 'anger',
 'joy',
 'joy',
 'joy',
 'joy',
 'anger',
 'joy',
 'joy',
 'joy',
 'joy',
 'joy',
 'sadness',
 'sadness',
 'joy',
 'love',
 'joy',
 'anger',
 'joy',
 'sadness',
 'anger',
 'fear',
 'joy',
 'sadness',
 'sadness',
 'surprise',
 'joy',
 'joy',
 'joy',
 'love',
 'fear',
 'fear',
 'surprise',
 'anger',
 'anger',
 'sadness',
 'love',
 'joy',
 'sadness',
 'sadness',
 'joy',
 'sadness',
 'sadness',
 'sadness',
 'joy',
 'joy',
 'joy',
 'anger',
 'sadness',
 'anger',
 'anger',
 'anger',
 'joy',
 'joy',
 'joy',
 'joy',
 'sadness',
 'fear',
 'love',
 'anger',
 'sadness',
 'anger',
 'love',
 'sadness',
 'joy',
 'joy',
 'sadness',
 'anger',
 'love',
 'joy',
 '

In [35]:
labels=[]
for item in y:
        if item not in labels:
            labels.append(item)
labels 

['sadness', 'anger', 'love', 'surprise', 'fear', 'joy']

In [ ]:
## Text cleaning 
# 1. lower case convert all message 
# 2. a-z0-9   other characters 

# 4. Text cleaning function applying on each and every raw messages

In [37]:
def text_cleaning(sentences,stemming):
    cleaned_data = []
    for sentece in sentences: 
        message = sentece.lower()
        message = re.sub('[^a-z0-9 ]',"",message)
        ls_of_words = nltk.word_tokenize(message)
        ls_of_word_without_stopwords = [word for word in ls_of_words if word not in stopwords.words('english')]
        stemmed_words = [stemming.stem(word) for word in ls_of_word_without_stopwords ]
        message = " ".join(stemmed_words)
        cleaned_data.append(message)
    return cleaned_data

stem = PorterStemmer()
cleaned_data = text_cleaning(x,stemming=stem)
    

In [38]:
os.makedirs('models',exist_ok=True)
joblib.dump(cleaned_data,"./models/cleaned_data.lb")
print("successfully saved your cleaned data!")

successfully saved your cleaned data!


In [39]:
## trying  on a single message 
ls_of_stopwords = stopwords.words('english')
print(x[1])
single_message = x[1].lower()
single_message = re.sub('[^a-z0-9 ]',"",single_message)
ls_of_words = nltk.word_tokenize(single_message)  # single_message.split()
ls_of_word_without_stopwords = []
for word in ls_of_words: 
    if word not in ls_of_stopwords:
        ls_of_word_without_stopwords.append(word)
print(ls_of_word_without_stopwords)
stem = PorterStemmer()
stemmed_words = []
for word in ls_of_word_without_stopwords: 
    stemmed_words.append(stem.stem(word))
print(stemmed_words)
" ".join(stemmed_words)

i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake
['go', 'feeling', 'hopeless', 'damned', 'hopeful', 'around', 'someone', 'cares', 'awake']
['go', 'feel', 'hopeless', 'damn', 'hope', 'around', 'someon', 'care', 'awak']


'go feel hopeless damn hope around someon care awak'

# 5. Tokenaization

In [40]:
## Tokenaization 
tokenizer = Tokenizer(oov_token="<nothing>")
tokenizer.fit_on_texts(cleaned_data)
joblib.dump(tokenizer,'./models/tokenizer.lb')
print("successfully saved your tokenizer at this location : './models/tokenizer.lb'")

successfully saved your tokenizer at this location : './models/tokenizer.lb'


In [41]:
tokenizer.word_index   
# what assign to what word

{'<nothing>': 1,
 'feel': 2,
 'like': 3,
 'im': 4,
 'get': 5,
 'time': 6,
 'know': 7,
 'realli': 8,
 'make': 9,
 'go': 10,
 'want': 11,
 'love': 12,
 'littl': 13,
 'think': 14,
 'peopl': 15,
 'day': 16,
 'thing': 17,
 'one': 18,
 'would': 19,
 'even': 20,
 'still': 21,
 'ive': 22,
 'life': 23,
 'bit': 24,
 'way': 25,
 'need': 26,
 'someth': 27,
 'much': 28,
 'dont': 29,
 'work': 30,
 'start': 31,
 'could': 32,
 'say': 33,
 'look': 34,
 'see': 35,
 'tri': 36,
 'back': 37,
 'good': 38,
 'pretti': 39,
 'come': 40,
 'right': 41,
 'alway': 42,
 'help': 43,
 'also': 44,
 'today': 45,
 'year': 46,
 'take': 47,
 'friend': 48,
 'use': 49,
 'around': 50,
 'cant': 51,
 'person': 52,
 'made': 53,
 'though': 54,
 'hate': 55,
 'well': 56,
 'got': 57,
 'happi': 58,
 'thought': 59,
 'someon': 60,
 'didnt': 61,
 'never': 62,
 'felt': 63,
 'find': 64,
 'write': 65,
 'lot': 66,
 'hope': 67,
 'quit': 68,
 'live': 69,
 'week': 70,
 'everi': 71,
 'sure': 72,
 'less': 73,
 'read': 74,
 'enough': 75,
 'give':

In [42]:
tokenizer.word_counts
# it counts the occurance of words in text

OrderedDict([('didnt', 334),
             ('feel', 21204),
             ('humili', 69),
             ('go', 1101),
             ('hopeless', 81),
             ('damn', 54),
             ('hope', 320),
             ('around', 382),
             ('someon', 335),
             ('care', 254),
             ('awak', 22),
             ('im', 3055),
             ('grab', 22),
             ('minut', 79),
             ('post', 234),
             ('greedi', 79),
             ('wrong', 152),
             ('ever', 260),
             ('nostalg', 63),
             ('fireplac', 3),
             ('know', 1192),
             ('still', 743),
             ('properti', 5),
             ('grouchi', 35),
             ('ive', 723),
             ('littl', 932),
             ('burden', 95),
             ('late', 167),
             ('wasnt', 119),
             ('sure', 310),
             ('take', 403),
             ('milligram', 1),
             ('time', 1215),
             ('recommend', 19),
             ('amoun

In [43]:
tokenized_data = tokenizer.texts_to_sequences(cleaned_data)
tokenized_data
# change text msg to numerical data 

[[61, 2, 522],
 [10, 2, 419, 682, 67, 50, 60, 96, 1229],
 [4, 1230, 431, 107, 2, 432, 192],
 [92, 2, 592, 3696, 7, 21, 2844],
 [2, 918],
 [22, 2, 13, 343, 170, 260, 72],
 [22, 47, 6002, 6, 1347, 712, 22, 2375, 1348, 66, 1610, 44, 2, 3, 355],
 [2, 339, 23, 1174, 713, 46, 196, 298],
 [4452, 46, 2, 4452, 897, 56, 53, 887, 2845],
 [2, 550],
 [2, 3, 9, 373, 35, 139, 27],
 [2, 134, 508, 240, 337, 449, 1199, 1685],
 [14, 4453, 6, 46, 2, 523],
 [2, 299, 371, 3697],
 [1762,
  1499,
  106,
  154,
  308,
  6003,
  958,
  36,
  64,
  6,
  65,
  1349,
  23,
  1142,
  2846,
  93,
  137,
  1350,
  1500,
  2,
  13,
  472],
 [2, 638, 919, 377],
 [61, 8, 2, 325],
 [2, 39, 496, 6],
 [31, 2, 786, 2082, 311, 514, 1269, 2083, 3698, 2082, 4454],
 [2, 2376, 759, 320, 71, 2377, 30, 111],
 [2, 160, 380, 90, 182, 80, 33, 80],
 [2,
  130,
  147,
  148,
  2217,
  43,
  2,
  787,
  1868,
  123,
  1351,
  1118,
  1017,
  143,
  103,
  524,
  959,
  17,
  639],
 [2, 760, 714],
 [113, 43, 4455, 4456, 89, 1143, 214, 2,

In [44]:
len(tokenized_data)

20000

In [45]:
cleaned_data[0]

'didnt feel humili'

In [46]:
tokenized_data[0]

[61, 2, 522]

In [48]:
len_of_message = []
for i in range(len(tokenized_data)):
        len_of_message.append(len(tokenized_data[i]))
        
print("maximum length is : ",max(len_of_message))
# maximum length

maximum length is :  35


In [49]:
print("maximum length is : ",max(list(map(len,tokenized_data))))

maximum length is :  35


In [50]:
sequences = pad_sequences(tokenized_data, maxlen=35, padding="post")
sequences
# padding by default "pre"

array([[  61,    2,  522, ...,    0,    0,    0],
       [  10,    2,  419, ...,    0,    0,    0],
       [   4, 1230,  431, ...,    0,    0,    0],
       ...,
       [   2,  194,  157, ...,    0,    0,    0],
       [ 328,    2,  175, ...,    0,    0,    0],
       [   2,    3,  916, ...,    0,    0,    0]])

In [51]:
labels=[]
for item in y:
        if item not in labels:
            labels.append(item)
label_dict = {label:i for i , label in enumerate(labels)}
label_dict

{'sadness': 0, 'anger': 1, 'love': 2, 'surprise': 3, 'fear': 4, 'joy': 5}

# 6. Label Encoding

In [52]:
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
# automation 

In [53]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y)

In [55]:
Y

array([4, 4, 0, ..., 2, 2, 2], dtype=int64)

In [56]:
label_encoder.classes_

array(['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'], dtype='<U8')

In [57]:
joblib.dump(label_encoder,'./models/label_encoder.lb')

['./models/label_encoder.lb']

In [58]:
sequences    # <====  X DATA  cleaned data 
Y            # <====  Y DATA label 

array([4, 4, 0, ..., 2, 2, 2], dtype=int64)

# 7. Train test splitting 

In [59]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(sequences,Y,test_size=0.15)

In [ ]:
## Model training
## Model define  
# ANN , CNN , RNN  
### logistic regression ,
### DTC , RDC , 
## machine learning algorithm   | ANN   | RNN 

# 8. Defining the model

In [61]:
#  LSTM ---> layer of rnn , long short term memory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  LSTM, Dense

model = Sequential([
    LSTM(units= 128, input_shape= (35, 1), return_sequences= True),
    LSTM(units= 64),
    Dense(units= 6, activation= "softmax")
])

model.compile(optimizer= "adam", loss= "sparse_categorical_crossentropy", metrics= ["accuracy"])

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [62]:
history = model.fit(x_train, y_train, epochs= 5, validation_data= (x_test,y_test))
history

Epoch 1/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.3307 - loss: 1.5885 - val_accuracy: 0.3033 - val_loss: 1.5779
Epoch 2/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.3254 - loss: 1.5799 - val_accuracy: 0.3337 - val_loss: 1.5867
Epoch 3/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 19s 36ms/step - accuracy: 0.3357 - loss: 1.5768 - val_accuracy: 0.3540 - val_loss: 1.5701
Epoch 4/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 18s 34ms/step - accuracy: 0.3338 - loss: 1.5751 - val_accuracy: 0.3513 - val_loss: 1.5697
Epoch 5/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 19s 36ms/step - accuracy: 0.3438 - loss: 1.5755 - val_accuracy: 0.3340 - val_loss: 1.5712


# 9. Prediction 

In [63]:
y_pred = model.predict(x_test)
y_pred
# These values are probabilities of the class
# highest value will give the tokenized value of the word

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


array([[0.13568793, 0.13201892, 0.29765013, 0.05739832, 0.34307703,
        0.03416771],
       [0.13107213, 0.12022763, 0.32963052, 0.07556561, 0.30596974,
        0.03753439],
       [0.1291304 , 0.12298232, 0.32229593, 0.06950278, 0.3196329 ,
        0.03645569],
       ...,
       [0.13009131, 0.12568383, 0.31452453, 0.06540813, 0.32864127,
        0.035651  ],
       [0.12924086, 0.12299431, 0.32216817, 0.06951764, 0.31961715,
        0.03646181],
       [0.1305884 , 0.12636828, 0.31239754, 0.06452154, 0.33063447,
        0.0354898 ]], dtype=float32)

In [64]:
y_pred[0]

array([0.13568793, 0.13201892, 0.29765013, 0.05739832, 0.34307703,
       0.03416771], dtype=float32)

In [67]:
y_test[0]  # sadness

0

In [72]:
prediction = np.argmax(y_pred, axis= 1)
prediction

array([4, 2, 2, ..., 4, 2, 4], dtype=int64)

In [73]:
y_test

array([0, 3, 5, ..., 3, 3, 4], dtype=int64)

In [70]:
y_pred.shape

(3000, 6)

# 10. Creating dataframe for y_test and y_pred

In [76]:
df = pd.DataFrame({"actual": y_test, "prediction": prediction})
df

,actual,prediction
0,0,4
1,3,2
2,5,2
3,0,2
4,2,2
...,...,...
2995,2,4
2996,1,2
2997,3,4
2998,3,2


In [80]:
label_dictionary = {value:key for key, value in label_dict.items()}
label_dictionary

{0: 'sadness', 1: 'anger', 2: 'love', 3: 'surprise', 4: 'fear', 5: 'joy'}

# 11. Exchanging keys and values and their mapping data frame

In [81]:
df["actual"] = df["actual"].map(label_dictionary)
df["prediction"] = df["prediction"].map(label_dictionary)

In [82]:
df.head(40)

,actual,prediction
0,sadness,fear
1,surprise,love
2,joy,love
3,sadness,love
4,love,love
5,fear,fear
6,anger,fear
7,anger,fear
8,love,love
9,fear,fear


# 12. Defining model using Sequential SimpleRNN 

In [84]:
### using RNN 
from tensorflow.keras.layers import SimpleRNN
model = Sequential()
model.add(SimpleRNN(32,input_shape=(35,1),return_sequences=False))
model.add(Dense(6,activation='softmax'))



## compiling the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()
# return_sequence = False  --> means if you don't want to stack again this model then specify False

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,286 (5.02 KB)

 Trainable params: 1,286 (5.02 KB)

 Non-trainable params: 0 (0.00 B)

# 13. Saving model

In [85]:
model.save("./models/lstm_model.h5")
# h5 ----> extension for tensorflow

# 14. To load tensorflow model

In [91]:
from tensorflow.keras.models import load_model
loaded_model = load_model("./models/lstm_model.h5")
loaded_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,288 (5.04 KB)

 Trainable params: 1,286 (5.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
# matrix
# report